# 使用 functools.partial 批量应用正则化

## 问题背景

当需要对多个层应用相同的正则化配置时，重复代码会变得冗长且难以维护。

## 解决方案

使用 Python 标准库 `functools.partial` 创建预配置的层工厂函数。

In [ ]:
import tensorflow as tf
from tensorflow import keras
from functools import partial
import numpy as np
import matplotlib.pyplot as plt

# 设置随机种子
tf.random.set_seed(42)
np.random.seed(42)

print(f"TensorFlow 版本: {tf.__version__}")

## 1. 使用 partial 创建预配置层

In [ ]:
# 创建带 L2 正则化的 Dense 层工厂
# partial 会预先填充指定的参数
RegularizedDense = partial(
    keras.layers.Dense,
    kernel_regularizer=keras.regularizers.l2(0.001),
    activation='elu',
    kernel_initializer='he_normal'
)

# 使用方式与普通 Dense 相同，只需指定 units
layer1 = RegularizedDense(256)
layer2 = RegularizedDense(128)
layer3 = RegularizedDense(64)

print("预配置层创建成功")
print(f"默认配置: activation='elu', L2=0.001, initializer='he_normal'")

## 2. 构建完整模型

In [ ]:
# 使用 partial 构建模型
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.BatchNormalization(),
    
    # 使用预配置的 RegularizedDense
    RegularizedDense(256),
    RegularizedDense(128),
    RegularizedDense(64),
    
    # 输出层覆盖默认配置
    RegularizedDense(
        10, 
        activation='softmax',
        kernel_initializer='glorot_uniform'  # 覆盖默认初始化
    )
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

## 3. 对比传统写法

In [ ]:
# 传统写法（冗长）
model_traditional = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(256, activation='elu', kernel_regularizer=keras.regularizers.l2(0.001), kernel_initializer='he_normal'),
    keras.layers.Dense(128, activation='elu', kernel_regularizer=keras.regularizers.l2(0.001), kernel_initializer='he_normal'),
    keras.layers.Dense(64, activation='elu', kernel_regularizer=keras.regularizers.l2(0.001), kernel_initializer='he_normal'),
    keras.layers.Dense(10, activation='softmax', kernel_regularizer=keras.regularizers.l2(0.001), kernel_initializer='glorot_uniform')
])

print("传统写法代码行更长，且难以统一修改正则化参数")

## 4. 实战训练

In [ ]:
# 加载数据
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0

# 训练模型
history = model.fit(
    X_train, y_train,
    epochs=15,
    batch_size=64,
    validation_data=(X_valid, y_valid),
    verbose=1
)

# 评估
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n测试集准确率: {test_acc:.4f}")

## 5. 更多 partial 应用场景

In [ ]:
# 场景1: 带 Dropout 的标准隐藏层
HiddenLayerWithDropout = partial(
    keras.layers.Dense,
    activation='relu',
    kernel_initializer='he_normal',
    kernel_regularizer=keras.regularizers.l2(0.001)
)

# 场景2: 用于 CNN 的卷积层
RegularizedConv2D = partial(
    keras.layers.Conv2D,
    kernel_size=(3, 3),
    padding='same',
    activation='relu',
    kernel_regularizer=keras.regularizers.l2(0.0001)
)

# 场景3: 自归一化网络层
SelfNormalizingDense = partial(
    keras.layers.Dense,
    activation='selu',
    kernel_initializer='lecun_normal'
)

print("partial 可以应用于各种层类型")
print("优势: 集中管理配置，一处修改全局生效")

In [ ]:
# 验证代码正确性
print("partial 正则化模块测试完成")
print("\n关键要点:")
print("1. functools.partial 预配置层参数，减少代码重复")
print("2. 可以在使用时覆盖预设参数")
print("3. 便于统一管理和修改正则化配置")